<img width="8%" alt="LinkedIn.png" src="https://raw.githubusercontent.com/jupyter-naas/awesome-notebooks/master/.github/assets/logos/LinkedIn.png" style="border-radius: 15%">

# LinkedIn - Accept all invitations and send first message
<a href="https://bit.ly/3JyWIk6">Give Feedback</a> | <a href="https://github.com/jupyter-naas/awesome-notebooks/issues/new?assignees=&labels=bug&template=bug_report.md&title=LinkedIn+-+Accept+all+invitations+and+send+first+message:+Error+short+description">Bug report</a>

**Tags:** #linkedin #content #operations #automation #invitation

**Author:** [Florent Ravenel](https://www.linkedin.com/in/ACoAABCNSioBW3YZHc2lBHVG0E_TXYWitQkmwog/)

**Last update:** 2023-11-30 (Created: 2022-04-05)

**Description:** This notebook helps you quickly and easily accept all LinkedIn invitations and send a personalized introductory message to each new connection.


<div class="alert alert-info" role="info" style="margin: 10px">
<b>Disclaimer:</b><br>
This code is in no way affiliated with, authorized, maintained, sponsored or endorsed by Linkedin or any of its affiliates or subsidiaries. It uses an independent and unofficial API. Use at your own risk.

This project violates Linkedin's User Agreement Section 8.2, and because of this, Linkedin may (and will) temporarily or permanently ban your account. We are not responsible for your account being banned.
<br>
</div>

## Input

### Import libraries


In [ ]:
import naas
from naas_drivers import linkedin
import pandas as pd

### Setup variables
**Pre-requisite**

[Get your cookies on LinkedIn](https://www.notion.so/LinkedIn-driver-Get-your-cookies-d20a8e7e508e42af8a5b52e33f3dba75)

**Mandatory**
- `li_at`: Cookie used to authenticate Members and API clients
- `JSESSIONID`: Cookie used for Cross Site Request Forgery (CSRF) protection and URL signature validation

**Optional**
- `limit`: Number of invitations to be accepted. If limit = -1, all invitations are accepted.
- `first_message`: First message to be send
- `cron`: Cron params for naas scheduler. More information: https://crontab.guru/

In [ ]:
# Mandatory
li_at = naas.secret.get("LINKEDIN_LI_AT") or "YOUR_COOKIE_LI_AT"
JSESSIONID = naas.secret.get("LINKEDIN_JSESSIONID") or "YOUR_COOKIE_JSESSIONID"

# Optional
limit = -1
first_message = "Hello, Nice to connect!"
cron = "0 18 * * *"

## Model

### Get invitations received

In [ ]:
df_invitation = linkedin.connect(li_at, JSESSIONID).invitation.get_received()
print('Invitations received:', len(df_invitation))
df_invitation

### Accept pending invitations received from "Profile"

In [ ]:
def accept_new_contact(df, limit):
    # Init
    df_accept = pd.DataFrame()
    if len(df) == 0:
        return df_accept

    # Loop
    for index, row in df.iterrows():
        fullname = row.FULLNAME
        status = row.INVITATION_STATUS
        invitation_id = row.INVITATION_ID
        shared_secret = row.SHARED_SECRET
        if status == "PENDING":
            print(fullname)
            tmp_df = linkedin.connect(li_at, JSESSIONID).invitation.accept(
                invitation_id, shared_secret
            )
            df_accept = pd.concat([df_accept, tmp_df])
        if index != -1 and index + 1 >= limit:
            break
    return df_accept

df_accept = accept_new_contact(df_invitation, limit)
print('Invitations accepted:', len(df_accept))
df_accept

### Send first message to contact

In [ ]:
def send_first_message(df):
    if len(df) > 0:
        # Loop
        for index, row in df.iterrows():
            fullname = row.FULLNAME
            profile_id = row.PROFILE_ID
            print("Message sent to:", fullname)
            linkedin.connect(li_at, JSESSIONID).message.send(FIRST_MESSAGE, profile_id)

send_first_message(df_accept)

## Output


### Display result


In [ ]:
df_accept

### Add scheduler

In [ ]:
# Schedule your notebook every hour
naas.scheduler.add(cron=cron)

# -> Uncomment the line below to remove your scheduler
# naas.scheduler.delete()